# Blending MODIS LST with CFSv2 Anomalies using Google Earth Engine (GEE)
This notebook demonstrates how to blend MODIS Land Surface Temperature (LST) data with CFSv2 temperature anomalies to create a continuous LST dataset. The method uses Google Earth Engine (GEE) to perform data processing and blending.

## Step 1: Import Required Libraries
We begin by importing the necessary libraries for working with GEE, handling dates, and downloading data.


In [ ]:
# Import necessary libraries
import ee
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.metrics import r2_score, mean_squared_error
import requests

# Initialize the Earth Engine module
ee.Initialize()


## Step 2: Define the Blending Function
The `contLST` function blends MODIS LST data with CFSv2 temperature anomalies, performs gap filling, and returns the processed data for a given day and geometry.


In [ ]:
def contLST(day, geometry):
    # Define the images and collections
    CFSV2_TFA = ee.Image("users/shilosh/CFSv2_TFA_Daily_Global")
    MODIS_TFA = ee.Image("users/shilosh/MODIS_TFA_Daily_Global")
    
    # Set the date range
    firstDay = day
    lastDay = ee.Date(day).advance(1, 'day')
    
    Temperature_Band = 'Maximum_temperature_height_above_ground_6_Hour_Interval'
    Day_Temperature_Band = 'LST_Day_1km'
    Night_Temperature_Band = 'LST_Night_1km'
    collection = 'NOAA/CFSV2/FOR6H'
    
     # Apply scaling factor to MODIS data
    MODIS_TFA = MODIS_TFA.multiply(0.02)
    
    # Create image collections from the MODIS and CFSv2 bands
    MODIS_TFA_ic = ee.ImageCollection(MODIS_TFA.bandNames().map(lambda name: 
        MODIS_TFA.select([ee.String(name)], ['mod']).set('system:DOY', ee.Number.parse(ee.String(name).replace('TFA','0').replace('_','')).add(1))
    ))
    
    CFSV2_TFA_ic = ee.ImageCollection(CFSV2_TFA.bandNames().map(lambda name: 
        CFSV2_TFA.select([ee.String(name)], ['cfs']).set('system:DOY', ee.Number.parse(ee.String(name).replace('TFA','0').replace('_','')).add(1))
    ))
    
     # Resample and reproject images
    modisProjection = MODIS_TFA.projection().crs().getInfo()
    scale = ee.Image(MODIS_TFA).projection().nominalScale().getInfo()
    
    # Function to resample images
    def resample(image):
        return image.resample('bilinear').reproject(crs=modisProjection, scale=scale).set('system:DOY', image.get('system:DOY')).set('system:time_start', image.get('system:time_start'))
    
    # Convert Kelvin to Celsius
    def k2celsius(image):
        return image.subtract(ee.Image(273.15)).clip(geometry).set('system:DOY', image.get('system:DOY')).set('system:time_start', image.get('system:time_start'))

    # Add a property with doy to the collection
    def createDoyBand(img):
        d = ee.Date(img.get('system:time_start')).getRelative('day', 'year').add(1)
        return img.set('system:DOY', d)
    
    # Add timestamp to CFSv2
    def addTimeStampToCFSv2(image):
        start = ee.String(image.get('system:index'))
        y = start.slice(0, 4)
        m = start.slice(4, 6)
        d = start.slice(6, 8)
        date = y.cat('-').cat(m).cat('-').cat(d)
        return image.set({'system:time_start': date})
    
    # Add timestamp to MODIS
    def addTimeStampToMODIS(image):
        start = ee.String(image.get('system:index')).replace('_', '-')
        date = start.replace('_', '-')
        return image.set({'system:time_start': ee.String(date)})
    
    # Calculate the daily mean of the 4 images (00, 06, 12, 18)
    def daily_mean(image):
        return image.reduce(ee.Reducer.mean()).set('system:DOY', image.get('system:DOY')).set('system:time_start', image.get('system:time_start'))
    
    CFSV2_TFA_ic = CFSV2_TFA_ic.map(resample)
    
    # Convert the date string into milliseconds integer
    dayMillis = 86400000  # 86400000 is 1 day in milliseconds
    intFirstDay = ee.Date(firstDay).millis()
    intLastDay = ee.Date(lastDay).millis().subtract(dayMillis)
    
    # Collect all 4 images of each day and create imageCollection from the daily mean
    CFSV2 = ee.ImageCollection(ee.List.sequence(intFirstDay, intLastDay, dayMillis).map(lambda day: 
        ee.ImageCollection('NOAA/CFSV2/FOR6H')
            .select('Maximum_temperature_height_above_ground_6_Hour_Interval')
            .filterDate(day, ee.Number(day).add(dayMillis))
            .map(resample)
            .map(k2celsius)
            .mean()
            .set({'system:DOY': ee.Date(day).getRelative('day', 'year').add(1)})
            .set({'system:time_start': ee.Date(day)})
    ))
    
    # Use an equals filter to specify how the collections match
    Filter = ee.Filter.equals(leftField='system:DOY', rightField='system:DOY')
    
    # Define the join
    innerJoin = ee.Join.inner('primary', 'secondary')
    
    # Join CFSV2 with CFSV2_TFA_ic by DOY
    CFSV2_JoinInner = innerJoin.apply(CFSV2, CFSV2_TFA_ic, Filter)
    
    # Calculate CFSv2 anomalies
    CFSV2_Anomalies = CFSV2_JoinInner.map(lambda f: 
        ee.Image(f.get('primary')).subtract(ee.Image(f.get('secondary')))
            .set('system:time_start', ee.Image(f.get('primary')).get('system:time_start'))
            .set('system:DOY', ee.Image(f.get('primary')).get('system:DOY'))
    )
    
    # Join MODIS_TFA_ic with CFSV2_Anomalies by DOY
    MODIS_JoinInner = innerJoin.apply(CFSV2_Anomalies, MODIS_TFA_ic, Filter)
    #print('MODIS_JoinInner = ' ,MODIS_JoinInner)
    
    # Calculate MODIS TFA Plus CFSv2 anomalies
    MODIS_Continuous = MODIS_JoinInner.map(lambda f: 
        ee.Image(f.get('primary')).divide(2).add(ee.Image(f.get('secondary')))
            .set('system:time_start', ee.Image(f.get('primary')).get('system:time_start'))
            .set('system:DOY', ee.Image(f.get('primary')).get('system:DOY'))
    )
    #print('MODIS_Continuous = ' ,MODIS_Continuous)

    Temperature_Band = 'Light_Day_1km'
    collection = 'MODIS/061/MYD11A1'
    
    # Convert Kelvin to Celsius for MODIS
    def modis_k2celsius(image):
        return image.updateMask(image.select(Day_Temperature_Band)).updateMask(image.select(Night_Temperature_Band)).reduce(ee.Reducer.mean()).rename(Temperature_Band).multiply(ee.Image(0.02)).subtract(ee.Image(273.15)).clip(geometry).set('system:time_start', ee.Date(image.get('system:time_start'))).rename(ee.String('daily_').cat(ee.String(image.get('system:time_start'))))
    
    MODIS_LST = ee.ImageCollection(collection).filterDate(firstDay, lastDay).select(Day_Temperature_Band, Night_Temperature_Band).map(addTimeStampToMODIS).map(modis_k2celsius)
    
    # Use an equals filter to specify how the collections match for 'system:time_start'
    Filter_time = ee.Filter.equals(leftField='system:time_start', rightField='system:time_start')

    
    # Join MODIS_LST with MODIS_TFA_plus_CFSV2_Anomalies by DOY
    MODIS_Blended_JoinInner = innerJoin.apply(MODIS_LST, MODIS_Continuous, Filter_time)

    # Blend the results to fill LST gaps
    MODIS_LST_Blended = MODIS_Blended_JoinInner.map(lambda f: 
        ee.Image(f.get('primary')).unmask(ee.Image(f.get('secondary'))))
    
    return MODIS_LST_Blended  


## Step 3: Download Blended LST Data for a Date Range
This section loops through the specified date range, calls the blending function, and downloads the blended LST images in GeoTIFF format.

In [ ]:
# Specify the start and end dates
start_date = datetime.strptime('2015-04-01', '%Y-%m-%d')
end_date = datetime.strptime('2017-10-01', '%Y-%m-%d')

# Define the geometry (bounding box for West Africa)
geometry = ee.Geometry.Rectangle([-17.375775792219486, 4.332703063102031, 9.486975211670199, 15.866390948587537])

# Loop through each day in the date range
current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime('%Y-%m-%d')
    date = current_date.strftime('%Y%m%d')
    try:
        day_result = contLST(date_str, geometry)  # Call blending function
        images = day_result.toList(day_result.size())
        image = ee.Image(images.get(0))  # Extract the first image
        
        # Prepare image for download
        url = image.getDownloadURL({
            'scale': 1000,
            'crs': 'EPSG:32630',
            'region': geometry,
            'format': 'GEO_TIFF'
        })
        
        # Download the image
        response = requests.get(url)
        with open(f'LST_{date}.tif', 'wb') as file:
            file.write(response.content)
            
    except Exception as e:
        print(f"Error for {date_str}: {e}")
    
    current_date += timedelta(days=1)


## Step 4: Conclusion
This script successfully blends MODIS LST data with CFSv2 anomalies and downloads the results as GeoTIFF files. The data can be further analyzed for different applications such as climate modeling or land surface temperature studies.
